[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/MASt3R-jupyter/blob/main/MASt3R_gradio_jupyter.ipynb)

In [ ]:
%cd /content
!git clone -b dev --recursive https://github.com/camenduru/mast3r
%cd /content/mast3r

%cd dust3r/croco/models/curope
!python setup.py build_ext --inplace

!mkdir -p /content/mast3r/checkpoints
!wget https://huggingface.co/camenduru/mast3r/resolve/main/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric.pth -O /content/mast3r/checkpoints/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric.pth

!pip install roma einops trimesh gradio
!pip install /content/mast3r/dust3r/croco/models/curope

%cd /content/mast3r
!python demo.py --share --weights /content/mast3r/checkpoints/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric.pth